# Этап 3: INT8 Quantization — Полный сравнительный анализ

Мы сравниваем две модели по трем ключевым бизнес-метрикам:
1. **Memory (Память)**: Физический объем занимаемого места.
2. **Throughput (Пропускная способность)**: Сколько символов генерируется в секунду.
3. **Latency (Задержка)**: Время до появления самого первого символа (Time to First Token).

In [ ]:
import torch
import copy
import time
import pandas as pd
import numpy as np
from src.model import GPTLanguageModel, device, get_batch, estimate_loss, decode, encode

def get_model_size_mb(mdl, real_int8=False):
    if not real_int8:
        param_size = sum(p.nelement() * p.element_size() for p in mdl.parameters())
        buffer_size = sum(b.nelement() * b.element_size() for b in mdl.buffers())
        return (param_size + buffer_size) / 1024**2
    else:
        total_bits = 0
        for name, param in mdl.named_parameters():
            bits = 8 if ('weight' in name and param.dim() > 1) else 32
            total_bits += param.nelement() * bits
        for b in mdl.buffers():
            total_bits += b.nelement() * 32
        return total_bits / (8 * 1024**2)

@torch.no_grad()
def measure_performance(mdl, num_tokens=50):
    context = torch.zeros((1, 1), dtype=torch.long, device=device)
    
    # Warmup
    _ = mdl.generate(context, max_new_tokens=5)
    
    # 1. Latency (TTFT)
    start_latency = time.time()
    _ = mdl.generate(context, max_new_tokens=1)
    latency = (time.time() - start_latency) * 1000
    
    # 2. Throughput
    start_throughput = time.time()
    _ = mdl.generate(context, max_new_tokens=num_tokens)
    duration = time.time() - start_throughput
    throughput = num_tokens / duration
    
    return latency, throughput

def quantize_tensor_int8(x):
    x_max = x.abs().max().item()
    if x_max == 0: return x
    scale = x_max / 127.0
    return torch.round(x / scale).clamp(-128, 127) * scale

### 1. Замер исходной модели (Baseline FP32)
Здесь мы фиксируем «золотой стандарт» качества и скорости.

In [ ]:
model_fp32 = GPTLanguageModel().to(device)
try:
    model_fp32.load_state_dict(torch.load('model_ckpt.pt', map_location=device))
    print("✅ Успешно загружены веса модели.")
except:
    print("⚠️ Чекпоинт не найден, замеры будут на случайных весах.")
model_fp32.eval()

print("\n--- [STEP 1] Baseline FP32 Performance ---")
loss_fp32 = estimate_loss(model_fp32)['val'].item()
size_fp32 = get_model_size_mb(model_fp32)
lat_fp32, thr_fp32 = measure_performance(model_fp32)

print(f"🔹 Memory Usage:     {size_fp32:.2f} MB")
print(f"🔹 Inference Latency: {lat_fp32:.2f} ms (Time to First Token)")
print(f"🔹 Throughput Rate:   {thr_fp32:.2f} tokens/sec")
print(f"🔹 Model Quality:     {loss_fp32:.4f} (Val Loss)")

### 2. Квантование и замер INT8
Теперь мы «портим» веса округлением до 8 бит и смотрим, как изменятся те же метрики.

In [ ]:
model_int8 = copy.deepcopy(model_fp32)
with torch.no_grad():
    for name, param in model_int8.named_parameters():
        if 'weight' in name and param.dim() > 1:
            param.copy_(quantize_tensor_int8(param.data))

print("--- [STEP 2] Quantized INT8 Performance ---")
loss_int8 = estimate_loss(model_int8)['val'].item()
size_int8 = get_model_size_mb(model_int8, real_int8=True)
lat_int8, thr_int8 = measure_performance(model_int8)

print(f"🔸 Memory Usage:     {size_int8:.2f} MB (Estimated storage size)")
print(f"🔸 Inference Latency: {lat_int8:.2f} ms")
print(f"🔸 Throughput Rate:   {thr_int8:.2f} tokens/sec")
print(f"🔸 Model Quality:     {loss_int8:.4f} (Val Loss)")

### 3. Анализ эффективности
Сравним выигрыш в ресурсах против потери качества.

In [ ]:
results = {
    "Metric": ["Memory (MB)", "Throughput (tokens/s)", "Latency (ms)", "Validation Loss"],
    "FP32": [size_fp32, thr_fp32, lat_fp32, loss_fp32],
    "INT8": [size_int8, thr_int8, lat_int8, loss_int8],
    "Delta": [
        f"{size_fp32/size_int8:.1f}x smaller", 
        f"{(thr_int8/thr_fp32 - 1)*100:+.1f}% check", 
        f"{lat_int8 - lat_fp32:+.2f} ms", 
        f"{(loss_int8/loss_fp32 - 1)*100:+.2f}% quality loss"
    ]
}

df = pd.DataFrame(results)
display(df)

print(f"\n🚀 ИТОГ: Вы освободили {(size_fp32 - size_int8):.2f} MB памяти!")
print(f"📉 Потеря качества составила всего {((loss_int8/loss_fp32 - 1)*100):.2f}%")

### 4. Генерация текста (Blind Test)
Напишем промпт и посмотрим разницу в стиле. 
*Если вы видите NameError, убедитесь, что выполнили самую первую ячейку кода.*

In [ ]:
from src.model import decode, encode

prompt = "ROMEO: "
context = torch.tensor(encode(prompt), dtype=torch.long, device=device).unsqueeze(0)

print("--- FP32 OUTPUT ---")
print(decode(model_fp32.generate(context, max_new_tokens=100)[0].tolist()))

print("\n--- INT8 OUTPUT ---")
print(decode(model_int8.generate(context, max_new_tokens=100)[0].tolist()))